<a href="https://colab.research.google.com/github/Oliver-Shariff/Pro-Image-Sports-ETL-Scripts/blob/main/Pro_Image_Sports_Deposit_Calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
from datetime import datetime
import pandas as pd

In [ ]:
# Load the XLSX File
uploaded = files.upload()
file_path = list(uploaded.keys())[0]

In [ ]:
# Read file
temp_df = pd.read_excel(file_path, skiprows=2)

# Calculate drawer_entry_sum from the loaded data
# Select the 'Error ($)' column, convert to string, filter for values starting with '$' or '-$'
dollar_values_str = temp_df['Error ($)'].astype(str)[temp_df['Error ($)'].astype(str).str.startswith(('$', '-$'), na=False)]

# Remove the '$' sign and convert to numeric
dollar_values_numeric = pd.to_numeric(dollar_values_str.str.replace('$', '', regex=False), errors='coerce')

# Calculate the sum and store in drawer_entry_sum
drawer_entry_sum = dollar_values_numeric.sum()

print(f"Sum of drawer entries starting with '$' or '-$': {drawer_entry_sum:.2f}")


# Process the main DataFrame from the loaded data

# Filter rows: keep rows where the first column starts with "P"
# This removes extraneous rows caused by cash drawer operations
df = temp_df[temp_df.iloc[:, 0].astype(str).str.startswith('P', na=False)].copy() # Use .copy() to avoid SettingWithCopyWarning


# Rename columns for clarity
df = df.rename(columns={
    "Unnamed: 7": "Start cash ($)",
    "Unnamed: 9": "Error ($)",
    "Unnamed: 10": "Amt. left in drawer ($)",
    "Unnamed: 11": "Amt. to deposit ($)",
    "Unnamed: 4": "Closed By",
    "Unnamed: 5": "Closed on",
    "Unnamed: 3": "Opened on",
    "Unnamed: 2": "Opened By"

})


# Keep relevant numeric columns
df = df[["Closed By", "Closed on", "Opened on", "Opened By", "Start cash ($)", "Amt. left in drawer ($)", "Error ($)", "Amt. to deposit ($)"]]

# Convert numeric columns to numeric type
numeric_cols = ["Start cash ($)", "Amt. left in drawer ($)", "Error ($)", "Amt. to deposit ($)"]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')


print("\n=== Main DataFrame (cleaned) ===")
print(df)

In [ ]:
# Get cash sales for this period from Sales By Payment method report
# Enter this number manually in 0.00 format: no "$" or ","
# EX: 9726.59
# Remember to filter payment method to cash and to not include ongoing cash sales if midday
# https://my.franpos.com/SalesByPaymentTypeReport

user_input_str = input("Enter Cash Sales for this period: ")

# Convert the input string to a float
try:
    cash_sales = float(user_input_str)
    print(f"Converted to float: {cash_sales}")
except ValueError:
    print("Invalid input. Please enter a valid number for Cash Sales. Do not include commas or dollar signs")
    cash_sales = None # Assign None or handle the error as needed

In [ ]:
# Calculate stable_drawer
# This accounts for differences in drawer values over the reporting period

# Get the 'Start cash ($)' of the last row
last_row_start_cash = df.iloc[-1]["Start cash ($)"]

# Get the 'Amt. left in drawer ($)' and 'Start cash ($)' of the first row
first_row_amt_left = df.iloc[0]["Amt. left in drawer ($)"]
first_row_start_cash = df.iloc[0]["Start cash ($)"]

# Determine the value to use from the first row
if pd.isna(first_row_amt_left):
    first_row_value = first_row_start_cash
elif (first_row_amt_left == 0):
    first_row_value = first_row_start_cash
else:
    first_row_value = first_row_amt_left

# Calculate the difference
stable_drawer = last_row_start_cash - first_row_value
print(f"Stable drawer difference: {stable_drawer:.2f}")

In [ ]:
# Calculate Missing cash
df["Next day Start cash ($)"] = df["Start cash ($)"].shift(1)
df["Missing cash"] = (df["Amt. left in drawer ($)"] - df["Next day Start cash ($)"]).round(2)

# Calculate total expected deposit  vs total actual deposit

# Remove Summary row if it was already created on a previous execution
df = df[df["Closed By"] != "Cash sales"]

total_deposit = df["Amt. to deposit ($)"].sum().round(2)
total_cash_sales = cash_sales

# Calculate the expected deposit
expected_deposit = total_cash_sales + drawer_entry_sum + stable_drawer

# Create a new row for the total deposit
# Create a dictionary representing the new row
Summary_row = {
    "Closed By": "Cash sales",
    "Closed on": total_cash_sales,
    "Opened on": "drawer entries",
    "Opened By": drawer_entry_sum,
    "Start cash ($)": "expected deposit",
    "Amt. left in drawer ($)": expected_deposit,
    "Error ($)": "Total Deposits",
    "Amt. to deposit ($)": total_deposit,
    "Next day Start cash ($)": "Discrepancy",
    "Missing cash": expected_deposit - total_deposit
}

# Convert the dictionary to a DataFrame
Summary_df = pd.DataFrame([Summary_row])

# Concatenate the original DataFrame and the Summary row DataFrame
df = pd.concat([df, Summary_df], ignore_index=True)


# Output results
print( df[df["Closed By"] == "Cash sales"])

In [ ]:

# Export the result to a CSV file

# Get current date and time and format it
now = datetime.now()
timestamp = now.strftime("%m-%d-%Y_%I-%M %p")

# append timestamp to filename
output_filename = f"Cleaned_close_day_report_{timestamp}.csv"
df.to_csv(output_filename, index=False)

print(f"DataFrame successfully exported to '{output_filename}'")

# Download the file
files.download(output_filename)